In [1]:
# Imports
import warnings
import time
import gc
import os
import pandas as pd
import numpy as np
from IPython.display import HTML, display

# Visualization options
pd.set_option('display.float_format', lambda x: '%.2f' % x)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
display(HTML("<style>.container { width: 90% !important; }</style>"))
warnings.filterwarnings("ignore")

# Global variables
SEED = 2017
N_JOBS = 18


<b>Functions</b>

In [2]:
# Define error metric
def smape(y_true, y_pred):
    denominator = (y_true + np.abs(y_pred)) / 200.0
    diff = np.abs(y_true - y_pred) / denominator
    diff[denominator == 0] = 0.0
    return np.mean(diff)
    #return np.nanmean(diff)

In [3]:
# For each data configuration, get CV score and compute test set predictions
def run_generator(data_configs, trains, vals, init_train, test) :
    # Clear predictions directories
    #list(map(os.unlink, (os.path.join("../oof_preds/temp/",f) for f in os.listdir("../oof_preds/temp/"))))
    #list(map(os.unlink, (os.path.join("../oof_targets/temp/",f) for f in os.listdir("../oof_targets/temp/"))))
    #list(map(os.unlink, (os.path.join("../preds/temp/",f) for f in os.listdir("../preds/temp/"))))
    
    # Iterate on each data configuration :
    results = pd.DataFrame()
    for conf_name, conf in data_configs.items() :
        print("***** CONFIG : " + conf_name + " ***************")
        cur_train = init_train.copy()
        cur_test = test.copy()
        
        # Cross validation
        cv_score = cross_val(conf, conf_name, trains, vals)

        # Store results
        cur_result = pd.DataFrame(columns = ["conf", "cv"])
        cur_result.loc[0] = [conf_name, cv_score] 
        results = pd.concat([results, cur_result], axis = 0)
        
        '''# Predict test set
        start = time.time()
        cur_test["Date"] = cur_test.Page.apply(lambda a: a[-10:])
        cur_test["Page"] = cur_test.Page.apply(lambda a: a[:-11])
        print("Elapsed prepare test : " + str(time.time() - start))
        cur_test, y = fit_predict(conf, cur_train, cur_test, is_test=True)
   
        # Save preds
        file_name = "../preds/temp/" + conf_name + "_preds_test.csv"
        cur_test[["Id", "Visits"]].to_csv(file_name, index = False)'''
        
        del(cur_train)
        del(cur_test)
        gc.collect()

    return(results)
    

In [4]:
# Cross validation
def cross_val(conf, conf_name, trains, vals):
    start_a = time.time()
    oof_preds = pd.DataFrame()
    oof_targets = pd.DataFrame()
    for i in range(len(trains)):
        start_f = time.time()
        train = trains[i]
        validation = vals[i]

        validation, y = fit_predict(conf, train, validation, is_test=False)

        # Compute validation score
        val_score = smape(y.Visits, validation.Visits)
        print("SMAPE fold " + str(i + 1) + " : " + str(val_score))

        # Assemble OOF predictions and targets
        oof_preds = pd.concat([pd.Series(oof_preds), pd.Series(validation.Visits)], axis = 0)
        oof_targets = pd.concat([pd.Series(oof_targets), pd.Series(y.Visits)], axis = 0)

        print("Elapsed in fold " + str(i + 1) + " : " + str(time.time() - start_f))
        del(train)
        del(validation)
        gc.collect()

    # Compute overall score
    cv_score = smape(oof_targets, oof_preds)
    print("Global SMAPE score : " + str(cv_score))

    # Save OOF preds
    file_name = "../oof_preds/temp/" + conf_name + "_preds_OOF.csv"
    pd.DataFrame({"y": oof_preds}).to_csv(file_name, index = False)
    file_name = "../oof_targets/temp/" + conf_name + "_targets_OOF.csv"
    pd.DataFrame({"y": oof_targets}).to_csv(file_name, index = False)

    print("Elapsed cross val : " + str(time.time() - start_a))
    del(oof_targets)
    del(oof_preds)
    gc.collect()

    return(cv_score)

In [5]:
# Fit and predict
def fit_predict(conf, train, validation, is_test = False) :
    # Handle training length (in days)
    if (conf["length"] != "all") :
        train = train[["Page"] + list(train.columns[-conf["length"]:])]

    # Melt df to have one observation per row
    train = pd.melt(train, 
                    id_vars="Page", 
                    var_name="Date", 
                    value_name="Visits")
    #print(train.shape)
    #display(train.head(3))

    y = 0
    if (is_test is False) :
        validation = pd.melt(validation, 
                             id_vars="Page", 
                             var_name="Date", 
                             value_name="Visits")  
        
        # Remove y from validation set
        y = validation[["Page", "Visits"]]
        validation.drop("Visits", axis=1, inplace=True)
        #display(validation.head(3))
        #print(validation.shape)
        gc.collect()

    # Handle date variable
    #start = time.time()
    train["Year"] = train["Date"].map(lambda x: int(str(x)[2:4]))
    train["Month"] = train["Date"].map(lambda x: int(str(x)[5:7]))
    train["DayOfM"] = train["Date"].map(lambda x: int(str(x)[8:10]))
    train["Date"] = train["Date"].astype("datetime64[ns]")
    train["DayOfY"] = train.Date.dt.dayofyear
    train["DayOfW"] = train.Date.dt.dayofweek
    train["is_wknd"] = (train.Date.dt.dayofweek >= 5).astype(int)
    train.drop(["Date"], inplace=True, axis=1)
    #display(train.head(3))
    #print("Elapsed train date : " + str(time.time() - start))

    #start = time.time()
    validation["Year"] = validation["Date"].map(lambda x: int(str(x)[2:4]))
    validation["Month"] = validation["Date"].map(lambda x: int(str(x)[5:7]))
    validation["DayOfM"] = validation["Date"].map(lambda x: int(str(x)[8:10]))
    validation["Date"] = validation["Date"].astype("datetime64[ns]")
    validation["DayOfY"] = validation.Date.dt.dayofyear
    validation["DayOfW"] = validation.Date.dt.dayofweek
    validation["is_wknd"] = (validation.Date.dt.dayofweek >= 5).astype(int)
    validation.drop(["Date"], inplace=True, axis=1)
    #display(validation.head(3))
    #print("Elapsed val date : " + str(time.time() - start))
    
    # Handle transformation
    if (conf["stat"] != "median") :
        pages_groups = train.groupby(["Page"]).median().reset_index()
        pages_groups = pages_groups[["Page", "Visits"]]
    elif (conf["stat"] != "mean") :
        pages_groups = train.groupby(["Page"]).mean().reset_index()
        pages_groups = pages_groups[["Page", "Visits"]]

    # Make validation predictions
    validation = validation.merge(pages_groups, how="left")
    #display(validation.head(3))
    #display(validation.tail(3))
    del(pages_groups)
    gc.collect()

    # Handle NAs
    validation.loc[validation.Visits.isnull(), "Visits"] = 0
    
    return(validation, y)


<b>Config</b>

In [6]:
dc1 = {"stat" : "median",
       "length" : 14}
dc2 = {"stat" : "median",
       "length" : 28}
dc3 = {"stat" : "mean",
       "length" : 14}
dc4 = {"stat" : "mean",
       "length" : 28}
dc5 = {"stat" : "median",
       "length" : 56} # 8 weeks
dc6 = {"stat" : "mean",
       "length" : 56}
dc7 = {"stat" : "median", 
       "length" : 84} # 12 weeks
dc8 = {"stat" : "mean",
       "length" : 84}
dc9 = {"stat" : "median", 
       "length" : 112} # 16 weeks
dc10 = {"stat" : "mean",
       "length" : 112}
dc11 = {"stat" : "median", 
       "length" : 168} # 24 weeks (6 * 4 weeks)
dc12 = {"stat" : "mean",
       "length" : 168}

data_configs = {
    "dc11" : dc11,
    "dc12" : dc12,
}


<b>Script</b>

In [7]:
# Get data
init_train = pd.read_csv("../raw_data/train_1.csv")
display(init_train.shape)
display(init_train.head(3))

test = pd.read_csv("../raw_data/key_1.csv")
display(test.shape)
display(test.head(3))



(145063, 551)

,Page,2015-07-01,2015-07-02,2015-07-03,2015-07-04,2015-07-05,2015-07-06,2015-07-07,2015-07-08,2015-07-09,2015-07-10,2015-07-11,2015-07-12,2015-07-13,2015-07-14,2015-07-15,2015-07-16,2015-07-17,2015-07-18,2015-07-19,2015-07-20,2015-07-21,2015-07-22,2015-07-23,2015-07-24,2015-07-25,2015-07-26,2015-07-27,2015-07-28,2015-07-29,2015-07-30,2015-07-31,2015-08-01,2015-08-02,2015-08-03,2015-08-04,2015-08-05,2015-08-06,2015-08-07,2015-08-08,2015-08-09,2015-08-10,2015-08-11,2015-08-12,2015-08-13,2015-08-14,2015-08-15,2015-08-16,2015-08-17,2015-08-18,2015-08-19,2015-08-20,2015-08-21,2015-08-22,2015-08-23,2015-08-24,2015-08-25,2015-08-26,2015-08-27,2015-08-28,2015-08-29,2015-08-30,2015-08-31,2015-09-01,2015-09-02,2015-09-03,2015-09-04,2015-09-05,2015-09-06,2015-09-07,2015-09-08,2015-09-09,2015-09-10,2015-09-11,2015-09-12,2015-09-13,2015-09-14,2015-09-15,2015-09-16,2015-09-17,2015-09-18,2015-09-19,2015-09-20,2015-09-21,2015-09-22,2015-09-23,2015-09-24,2015-09-25,2015-09-26,2015-09-27,2015-09-28,2015-09-29,2015-09-30,2015-10-01,2015-10-02,2015-10-03,2015-10-04,2015-10-05,2015-10-06,2015-10-07,2015-10-08,2015-10-09,2015-10-10,2015-10-11,2015-10-12,2015-10-13,2015-10-14,2015-10-15,2015-10-16,2015-10-17,2015-10-18,2015-10-19,2015-10-20,2015-10-21,2015-10-22,2015-10-23,2015-10-24,2015-10-25,2015-10-26,2015-10-27,2015-10-28,2015-10-29,2015-10-30,2015-10-31,2015-11-01,2015-11-02,2015-11-03,2015-11-04,2015-11-05,2015-11-06,2015-11-07,2015-11-08,2015-11-09,2015-11-10,2015-11-11,2015-11-12,2015-11-13,2015-11-14,2015-11-15,2015-11-16,2015-11-17,2015-11-18,2015-11-19,2015-11-20,2015-11-21,2015-11-22,2015-11-23,2015-11-24,2015-11-25,2015-11-26,2015-11-27,2015-11-28,2015-11-29,2015-11-30,2015-12-01,2015-12-02,2015-12-03,2015-12-04,2015-12-05,2015-12-06,2015-12-07,2015-12-08,2015-12-09,2015-12-10,2015-12-11,2015-12-12,2015-12-13,2015-12-14,2015-12-15,2015-12-16,2015-12-17,2015-12-18,2015-12-19,2015-12-20,2015-12-21,2015-12-22,2015-12-23,2015-12-24,2015-12-25,2015-12-26,2015-12-27,2015-12-28,2015-12-29,2015-12-30,2015-12-31,2016-01-01,2016-01-02,2016-01-03,2016-01-04,2016-01-05,2016-01-06,2016-01-07,2016-01-08,2016-01-09,2016-01-10,2016-01-11,2016-01-12,2016-01-13,2016-01-14,2016-01-15,2016-01-16,2016-01-17,2016-01-18,2016-01-19,2016-01-20,2016-01-21,2016-01-22,2016-01-23,2016-01-24,2016-01-25,2016-01-26,2016-01-27,2016-01-28,2016-01-29,2016-01-30,2016-01-31,2016-02-01,2016-02-02,2016-02-03,2016-02-04,2016-02-05,2016-02-06,2016-02-07,2016-02-08,2016-02-09,2016-02-10,2016-02-11,2016-02-12,2016-02-13,2016-02-14,2016-02-15,2016-02-16,2016-02-17,2016-02-18,2016-02-19,2016-02-20,2016-02-21,2016-02-22,2016-02-23,2016-02-24,2016-02-25,2016-02-26,2016-02-27,2016-02-28,2016-02-29,2016-03-01,2016-03-02,2016-03-03,2016-03-04,2016-03-05,2016-03-06,2016-03-07,2016-03-08,2016-03-09,2016-03-10,2016-03-11,2016-03-12,2016-03-13,2016-03-14,2016-03-15,2016-03-16,2016-03-17,2016-03-18,2016-03-19,2016-03-20,2016-03-21,2016-03-22,2016-03-23,2016-03-24,2016-03-25,2016-03-26,2016-03-27,2016-03-28,2016-03-29,2016-03-30,2016-03-31,2016-04-01,2016-04-02,2016-04-03,2016-04-04,2016-04-05,2016-04-06,2016-04-07,2016-04-08,2016-04-09,2016-04-10,2016-04-11,2016-04-12,2016-04-13,2016-04-14,2016-04-15,2016-04-16,2016-04-17,2016-04-18,2016-04-19,2016-04-20,2016-04-21,2016-04-22,2016-04-23,2016-04-24,2016-04-25,2016-04-26,2016-04-27,2016-04-28,2016-04-29,2016-04-30,2016-05-01,2016-05-02,2016-05-03,2016-05-04,2016-05-05,2016-05-06,2016-05-07,2016-05-08,2016-05-09,2016-05-10,2016-05-11,2016-05-12,2016-05-13,2016-05-14,2016-05-15,2016-05-16,2016-05-17,2016-05-18,2016-05-19,2016-05-20,2016-05-21,2016-05-22,2016-05-23,2016-05-24,2016-05-25,2016-05-26,2016-05-27,2016-05-28,2016-05-29,2016-05-30,2016-05-31,2016-06-01,2016-06-02,2016-06-03,2016-06-04,2016-06-05,2016-06-06,2016-06-07,2016-06-08,2016-06-09,2016-06-10,2016-06-11,2016-06-12,2016-06-13,2016-06-14,2016-06-15,2016-06-16,2016-06-17,2016-06-18,2016-06-19,2016-06-20,2016-06-21,2016-06-22,2016-06-23,2016-06-24,2016-06-25,2016-06-26,2016-06-27,2

(8703780, 2)

,Page,Id
0,!vote_en.wikipedia.org_all-access_all-agents_2...,bf4edcf969af
1,!vote_en.wikipedia.org_all-access_all-agents_2...,929ed2bf52b9
2,!vote_en.wikipedia.org_all-access_all-agents_2...,ff29d0f51d5c


In [8]:
# Create validation data - 60 days val sets, 10 days between train and val
validation5 = init_train[["Page"] + list(init_train.columns[491:551])]
train5 = init_train[["Page"] + list(init_train.columns[1:481])]

validation4 = init_train[["Page"] + list(init_train.columns[431:491])]
train4 = init_train[["Page"] + list(init_train.columns[1:421])]

validation3 = init_train[["Page"] + list(init_train.columns[371:431])]
train3 = init_train[["Page"] + list(init_train.columns[1:361])]

validation2 = init_train[["Page"] + list(init_train.columns[311:371])]
train2 = init_train[["Page"] + list(init_train.columns[1:301])]

validation1 = init_train[["Page"] + list(init_train.columns[251:311])]
train1 = init_train[["Page"] + list(init_train.columns[1:241])]

trains = [train1, train2, train3, train4, train5]
vals = [validation1, validation2, validation3, validation4, validation5]
gc.collect()

27

In [9]:
# Generate preds
results = run_generator(data_configs, trains, vals, init_train, test)

***** CONFIG : dc11 ***************
SMAPE fold 1 : 55.8237945678
Elapsed in fold 1 : 240.81540060043335
SMAPE fold 2 : 62.940635399
Elapsed in fold 2 : 237.71068382263184
SMAPE fold 3 : 69.1071397552
Elapsed in fold 3 : 238.89702725410461
SMAPE fold 4 : 65.2478760226
Elapsed in fold 4 : 240.39991188049316
SMAPE fold 5 : 65.8067628754
Elapsed in fold 5 : 239.20644426345825
Global SMAPE score : 63.8397972784
Elapsed cross val : 2044.7637009620667
***** CONFIG : dc12 ***************
SMAPE fold 1 : 48.9870190577
Elapsed in fold 1 : 241.87780904769897
SMAPE fold 2 : 50.9469663169
Elapsed in fold 2 : 240.80409622192383
SMAPE fold 3 : 53.4400846606
Elapsed in fold 3 : 240.12034559249878
SMAPE fold 4 : 50.3973908527
Elapsed in fold 4 : 242.08399438858032
SMAPE fold 5 : 52.1803124493
Elapsed in fold 5 : 242.20941591262817
Global SMAPE score : 51.204375353
Elapsed cross val : 2000.975704908371


In [10]:
# Show ordered results
results.sort_values(["cv"], ascending=True, inplace=True)
results.reset_index(drop=True, inplace=True)
display(results)

,conf,cv
0,dc12,51.20
1,dc11,63.84
